# Feeding the DeepL model 

based on: 

https://keras.io/preprocessing/image/

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

## Load Model

Imports and Data Preprocessing have to be executed, then the loaded model can be used for the Testing and Visualisation steps

In [26]:
# load modules 
import os
from keras.models import load_model

#specify directory
os.chdir('C:\\Users\\lsamsi\\Documents\\GitHub\\DLfM_BrandManagement')

In [27]:
model = load_model('./model/demo_model2.h5')

In [12]:
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_4 (Model)              (None, 204800)            23587712  
_________________________________________________________________
dense_9 (Dense)              (None, 64)                13107264  
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 5)                 325       
Total params: 36,695,301
Trainable params: 13,107,589
Non-trainable params: 23,587,712
_________________________________________________________________
None


## Load Data

In [30]:
#specify directory
import os
os.chdir('C:\\Users\\lsamsi\\Documents\\GitHub\\DLfM_BrandManagement')

In [31]:
import numpy as np
npimgs = np.load('data/instagram_images/sanpellegrino_official_npimgs.npy')

In [32]:
npimgs.shape

(12, 300, 300, 3)

## Feed Model

In [33]:
X_test = npimgs

In [34]:
y_pred = model.predict(X_test)
# there is not y_true, so we need to manually assess 
y_pred

array([[9.37585354e-01, 2.16888748e-02, 3.72633785e-02, 2.29412643e-03,
        1.16815243e-03],
       [6.44346893e-01, 2.66037315e-01, 1.96027700e-02, 3.72644886e-03,
        6.62865639e-02],
       [7.36011624e-01, 5.27325347e-02, 1.92534924e-01, 9.25113913e-03,
        9.46969353e-03],
       [1.11696348e-01, 8.82617652e-01, 2.00088107e-04, 3.51152325e-04,
        5.13476133e-03],
       [7.22232997e-01, 3.81023809e-02, 3.99103388e-03, 7.16907438e-03,
        2.28504479e-01],
       [3.96871090e-01, 2.59062145e-02, 5.08069575e-01, 1.86349433e-02,
        5.05181737e-02],
       [8.69442582e-01, 7.47352913e-02, 1.00508740e-03, 3.22890759e-04,
        5.44941388e-02],
       [5.41423261e-01, 1.16568111e-01, 1.18579924e-01, 4.69483174e-02,
        1.76480338e-01],
       [9.75925982e-01, 6.21557003e-03, 6.91989495e-04, 3.45157023e-04,
        1.68212987e-02],
       [8.21524024e-01, 1.13566354e-01, 3.47757936e-02, 1.91190559e-02,
        1.10147502e-02],
       [4.55198228e-01, 1.5838

In [17]:
# functions
def probabilty_to_classencoding(y_pred):
        """
        Output of softmax is probability distribution, 
        But we need the one hot encoding vector based on the class of 
        highest probabilty. 
        This function converts output of sigmoid to output class one hot encoding form
        """
        y_class = np.array( y_pred )
        idx = np.argmax(y_class, axis=-1)
        y_class = np.zeros(y_class.shape )
        y_class[ np.arange(y_class.shape[0]), idx] = 1
        return y_class
def binary_class_to_label(binary_class):
        """
        Return the class value based on the one hot encoding vector
        """
        return(np.argmax(binary_class, axis=1))   

In [18]:
from sklearn.preprocessing import LabelEncoder

# convert from encoded label to label name 
label_encoder = LabelEncoder()
y_pred = probabilty_to_classencoding(y_pred)
y_pred = binary_class_to_label(y_pred)
y_pred_labels = label_encoder.inverse_transform(y_pred)
print(y_pred_labels)

NotFittedError: This LabelEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
# statistics: frequency/probability distribution of each label for images 